<a href="https://colab.research.google.com/github/Method-for-Software-System-Development/Cloud_Computing/blob/develop/gui/FaultRepair_GUI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from IPython import get_ipython
from IPython.display import display
!pip install gradio --quiet
%pip install importnb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.1/54.1 MB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.1/323.1 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 20.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 1.6 MB/s eta 0:00:00


In [ ]:
import os, subprocess, sys

REPO_DIR = "/content/Cloud_Computing"
LOGIC_DIR = os.path.join(REPO_DIR, "logic")

# Clone the repo if it's not there yet
if not os.path.isdir(REPO_DIR):
    subprocess.run([
        "git", "clone", "-b", "develop",
        "https://github.com/Method-for-Software-System-Development/Cloud_Computing.git",
        REPO_DIR
    ], check=True)

# Pull latest from develop
subprocess.run(["git", "-C", REPO_DIR, "fetch", "origin"], check=True)
subprocess.run(["git", "-C", REPO_DIR, "checkout", "develop"], check=True)
subprocess.run(["git", "-C", REPO_DIR, "pull"], check=True)

# Install importnb (for notebook importing)
%pip install importnb --quiet

# Add logic folder to sys.path so we can import from it
sys.path.append(LOGIC_DIR)

# Import Fault_controller from logic/Fault_controller.ipynb
from importnb import Notebook
with Notebook():
    import Fault_controller as fc
    import repair_controller as fr

omer_cohen 200
shira_gold 200
daniel_bar 200
roni_benami 200
lior_levi 200
yael_elyashiv 200
noa_katz 200
matan_tal 200
or_peled 200
tom_segal 200
Indoor Sensor Data:
Status: 200 | Uploaded: {'Temperature': 26.4, 'Humidity': 41.0, 'Pressure': 974.1, 'Distance': 210.1}
Status: 200 | Uploaded: {'Temperature': 30.2, 'Humidity': 28.7, 'Pressure': 963.5, 'Distance': 8.9}
Status: 200 | Uploaded: {'Temperature': 36.5, 'Humidity': 91.3, 'Pressure': 919.8, 'Distance': 12.2}
Status: 200 | Uploaded: {'Temperature': 27.1, 'Humidity': 73.0, 'Pressure': 1050.4, 'Distance': 205.0}
Status: 200 | Uploaded: {'Temperature': 25.8, 'Humidity': 45.2, 'Pressure': 960.0, 'Distance': 210.0}
Status: 200 | Uploaded: {'Temperature': 32.7, 'Humidity': 29.5, 'Pressure': 940.2, 'Distance': 7.5}
Status: 200 | Uploaded: {'Temperature': 28.9, 'Humidity': 89.9, 'Pressure': 950.6, 'Distance': 250.0}
Status: 200 | Uploaded: {'Temperature': 30.0, 'Humidity': 70.1, 'Pressure': 960.1, 'Distance': 9.5}
Status: 200 | Uploaded:

ImportError: cannot import name 'FAULT_RULES' from 'FAULT_RULES' (/content/Cloud_Computing/logic/FAULT_RULES.ipynb)

In [ ]:

import gradio as gr
from IPython.display import display
# --- UI Logic ---
def render_latest_fault_card():
    faults = fc.fb.get_active_faults()
    if not faults:
        return "<div class='no-fault'>✅ No faults currently detected</div>"

    # Get the last fault by time
    sorted_faults = sorted(faults.items(), key=lambda x: x[1].get("timestamp", ""), reverse=True)
    fault_id, fault = sorted_faults[0]

    title = fault.get("title", "Unknown")
    severity = fault.get("severity", "Unknown")
    status = fault.get("status", "Unknown")
    steps = fault.get("steps", [])
    xp = fault.get("xp", 50)
    timestamp = title.split(" (")[-1].replace(")", "") if "(" in title else "Unknown"

    # Clean title for display
    display_title = title.split(" (")[0]

    steps_html = "".join(f"<li>{step}</li>" for step in steps)

    severity_color = {
        "High": "#f44336",
        "Medium": "#ff9800",
        "Low": "#4caf50"
    }.get(severity, "#9e9e9e")

    return f"""
    <div class="card">
      <div class="left" style="background-color:{severity_color};">
        <h3>🚨 NEW FAULT DETECTED</h3>
        <p><b>{timestamp}</b></p>
        <h2>{display_title}</h2>
        <p><b>Severity:</b> {severity}</p>
        <p><b>Status:</b> {status}</p>
        <p><b>Suggested Actions:</b></p>
        <ul>{steps_html}</ul>
      </div>
      <div class="right">
        <h3>🛠️ REPAIR CHALLENGE</h3>
        <p>Earn XP by fixing <b>{display_title}</b></p>
        <p>Repair Steps:</p>
        <ul>{steps_html}</ul>
        <div class="xp">🎯 {xp} XP</div>
        <button onclick="alert('Repair Started!')">Start Repair</button>
      </div>
    </div>
    """

In [ ]:

# --- Gradio UI ---
with gr.Blocks(css="""
.card {
  display: flex;
  width: 100%;
  border: 1px solid #ccc;
  border-radius: 16px;
  overflow: hidden;
  box-shadow: 0 4px 8px rgba(0,0,0,0.1);
  margin-top: 20px;
  font-family: 'Segoe UI', sans-serif;
}
.left, .right {
  padding: 20px;
  color: white;
}
.left {
  width: 60%;
}
.right {
  width: 40%;
  background: #f5f5f5;
  color: black;
}
.right button {
  padding: 10px 16px;
  background-color: #4caf50;
  color: white;
  border: none;
  border-radius: 6px;
  cursor: pointer;
}
.right button:hover {
  background-color: #43a047;
}
.right .xp {
  margin: 10px 0;
  font-size: 18px;
  font-weight: bold;
  color: #ff9800;
}
ul {
  padding-left: 20px;
}
.no-fault {
  font-size: 20px;
  padding: 20px;
  color: green;
}
""") as demo:
    gr.Markdown("## 🛠️ Latest Fault Overview")
    gr.HTML(render_latest_fault_card, every=5)

demo.launch()